In [80]:
import h5py

In [81]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
from pprint import pformat

from hloc import extract_features, match_features, localize_inloc, visualization

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Pipeline for indoor localization

## Setup
Here we declare the paths to the dataset, image pairs, and we choose the feature extractor and the matcher. You need to download the [InLoc dataset](https://www.visuallocalization.net/datasets/) and put it in `datasets/inloc/`, or change the path.

In [70]:
scene_name = 'i5noydFURQK' #8WUmhLawc2A, 'EDJbREhghzL', 'i5noydFURQK', 'jh4fc5c5qoQ', 'mJXqzFtmKg4'
# Set whether you want output for 2 queries or 2nd query or 1st query (small)
query_2 = '_only2query.txt'; queries_2 = '_2queries.txt'; small = '_small.txt'
h5_suffix = small

# change this if your dataset is somewhere else
dataset = Path('datasets/graphVPR/mp3d_' + scene_name + '_small/')
pairs = Path('pairs/graphVPR/mp3d_' + scene_name + '_small/')

outputs = Path('outputs/graphVPR/mp3d_' + scene_name + '_small/')  # where everything will be saved
# results = outputs / 'InLoc_hloc_superpoint+superglue_netvlad40.txt'  # the result file

In [71]:
#loc_pairs = pairs / 'pairs-query-netvlad40.txt'  # top 40 retrieved by NetVLAD
loc_pairs = pairs / ('pairs-query-mp3d_' + scene_name + h5_suffix)  # top 40 retrieved by NetVLAD

In [72]:
# list the standard configurations available
print(f'Configs for feature extractors:\n{pformat(extract_features.confs)}')
print(f'Configs for feature matchers:\n{pformat(match_features.confs)}')

Configs for feature extractors:
{'d2net-ss': {'model': {'multiscale': False, 'name': 'd2net'},
              'output': 'feats-d2net-ss',
              'preprocessing': {'grayscale': False, 'resize_max': 1600}},
 'dir': {'model': {'name': 'dir'},
         'output': 'global-feats-dir',
         'preprocessing': {'resize_max': 1024}},
 'netvlad': {'model': {'name': 'netvlad'},
             'output': 'global-feats-netvlad',
             'preprocessing': {'resize_max': 1024}},
 'sift': {'model': {'name': 'sift'},
          'output': 'feats-sift',
          'preprocessing': {'grayscale': True, 'resize_max': 1600}},
 'superpoint_aachen': {'model': {'max_keypoints': 4096,
                                 'name': 'superpoint',
                                 'nms_radius': 3},
                       'output': 'feats-superpoint-n4096-r1024',
                       'preprocessing': {'grayscale': True,
                                         'resize_max': 1024}},
 'superpoint_inloc': {'model': {'

In [73]:
# pick one of the configurations for extraction and matching
# you can also simply write your own here!
feature_conf = extract_features.confs['d2net-ss'] #superpoint_inloc
matcher_conf = match_features.confs['NN-mutual']
# matcher_conf = match_features.confs['superglue']

## Extract local features for database and query images

In [74]:
feature_path = extract_features.main(feature_conf, dataset, outputs)

[08/09/2021 12:51:00 INFO] Extracting local features with configuration:
{'model': {'multiscale': False, 'name': 'd2net'},
 'output': 'feats-d2net-ss',
 'preprocessing': {'grayscale': False, 'resize_max': 1600}}
[08/09/2021 12:51:00 INFO] Found 140 images in root datasets/graphVPR/mp3d_i5noydFURQK_small.


  0%|          | 0/140 [00:00<?, ?it/s]/home/shubodh/anaconda3/envs/hloc/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448224956/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
100%|██████████| 140/140 [05:12<00:00,  2.23s/it]

[08/09/2021 12:56:35 INFO] Finished exporting features.


In [78]:
feature_path

PosixPath('outputs/graphVPR/mp3d_i5noydFURQK_small/feats-d2net-ss.h5')

In [94]:
# Experimentation
with h5py.File(feature_path, 'r') as hfile:
    for key in hfile.keys():
        print(key)
        dset = hfile["/mp3d_query/i5noydFURQK/1_rgb-i5noydFURQK-bathroom1.png"]
        for key2 in dset.keys():
            print(key2)
            print(dset[key2])
#         print(dset)
#         matches0 = dset['matches0']
#         m0_np = np.array(matches0)
#         print(f"m0_np.shape {m0_np.shape}")

mp3d_query
descriptors
<HDF5 dataset "descriptors": shape (512, 7202), type "<f4">
image_size
<HDF5 dataset "image_size": shape (2,), type "<i8">
keypoints
<HDF5 dataset "keypoints": shape (7202, 2), type "<f4">
scores
<HDF5 dataset "scores": shape (7202,), type "<f4">
mp3d_reference
descriptors
<HDF5 dataset "descriptors": shape (512, 7202), type "<f4">
image_size
<HDF5 dataset "image_size": shape (2,), type "<i8">
keypoints
<HDF5 dataset "keypoints": shape (7202, 2), type "<f4">
scores
<HDF5 dataset "scores": shape (7202,), type "<f4">


## Match the query images
Here we assume that the localization pairs are already computed using image retrieval (NetVLAD). To generate new pairs from your own global descriptors, have a look at `hloc/pairs_from_retrieval.py`. These pairs are also used for the localization - see below.

In [76]:
match_path = match_features.main(matcher_conf, loc_pairs, feature_conf['output'], outputs)

[08/09/2021 12:56:35 INFO] Matching local features with configuration:
{'model': {'do_mutual_check': True, 'name': 'nearest_neighbor'},
 'output': 'matches-NN-mutual'}


100%|██████████| 882/882 [02:08<00:00,  6.85it/s]

[08/09/2021 12:58:44 INFO] Finished exporting matches.


In [77]:
match_path

PosixPath('outputs/graphVPR/mp3d_i5noydFURQK_small/feats-d2net-ss_matches-NN-mutual_pairs-query-mp3d_i5noydFURQK_small.h5')

## Localize!
Perform hierarchical localization using the precomputed retrieval and matches. Different from when localizing with Aachen, here we do not need a 3D SfM model here: the dataset already has 3D lidar scans. The file `InLoc_hloc_superpoint+superglue_netvlad40.txt` will contain the estimated query poses.

In [16]:
#localize_inloc.main(
#    dataset, loc_pairs, feature_path, match_path, results,
#    skip_matches=20)#20  # skip database images with too few matches

## Visualization
We parse the localization logs and for each query image plot matches and inliers with a few database images.

In [18]:
#visualization.visualize_loc(results, dataset, n=1, top_k_db=1, seed=2)